In [25]:
from numba import cuda, jit, vectorize
import numpy as np
import time
from numba import void, uint8 , uint32, uint64, int32, int64, float32, float64, f8

# Simple cuda algorithms

In [18]:
import math

@cuda.jit
def tanh(a):
    # Thread id
    tx = cuda.threadIdx.x
    # Block id
    ty = cuda.blockIdx.x
    # Block size
    bw = cuda.blockDim.x
    # THe actual id of thread on the grid
    pos = tx + ty * bw
    a[pos] = math.tanh(a[pos])

a = np.random.rand(10)
print(a)

tanh[32, 10](a)
a

[0.05702106 0.79560147 0.48281886 0.02902985 0.04606159 0.19667842
 0.96377143 0.88879281 0.7063751  0.16971963]


array([0.05695934, 0.66157056, 0.4484983 , 0.0290217 , 0.04602904,
       0.19418106, 0.7459544 , 0.71079697, 0.60839872, 0.16810861])

# Algorithms with reduce operation

### Use Build-in reduce
- https://numba.pydata.org/numba-doc/dev/cuda/reduction.html

In [3]:
import numpy
from numba import cuda

@cuda.reduce
def sum_reduce(a, b):
    return a + b

A = (numpy.arange(1234, dtype=numpy.float64)) + 1
expect = A.sum()      # numpy sum reduction
got = sum_reduce(A)   # cuda sum reduction
print(got)

761995.0


### Create your own reduce with cuda
- https://people.duke.edu/~ccc14/sta-663/CUDAPython.html

In [20]:
@cuda.jit('int32(int32, int32)', device=True)
def sum_device(a, b):
    return a + b


@cuda.jit(device=True)
def max_device(a, b):
    return a if a > b else b

@cuda.jit('void(int32[:], int32[:])')
def sum_reduce(a, b):
    "Simple implementation of reduction kernel"
    # Allocate static shared memory of 512 (max number of threads per block for CC < 3.0)
    # This limits the maximum block size to 512.
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    bw = cuda.blockDim.x
    sa = cuda.shared.array(shape=(512,), dtype=int32)

    i = tx + bx * bw

    sa[tx] = a[i]
    if tx == 0:
        s = sa[tx]
        cuda.syncthreads()
        for j in range(1, bw):
            s = sum_device(s, sa[j])
        b[bx] = s

# numbers to be added in the partial sum (must be less than or equal to 512)
# total length of vector to be summed
k = 4
n = 6*4 

a = np.random.randint(0, n, n).astype(np.int32)
print('a: ', a)
print('a.sum(): ', a.sum())

griddim = (k, 1)
blockdim = (a.size//k, 1)
print("Block dim: ", blockdim)
print("Grid dim: ", blockdim)


sum_reduce[griddim, blockdim](a, a)
print(a)
print("Partial sum: ", a[:k])
sum_reduce[1, griddim](a[:k], a)
print(a)
print("Sum: ", a[0])

a:  [ 4  7 13  0 18  1 20 18  1  8 17 21 10  0  2 10 19  9 15 12  6  5 16 21]
a.sum():  253
Block dim:  (6, 1)
Grid dim:  (6, 1)
[43 85 50 75 18  1 20 18  1  8 17 21 10  0  2 10 19  9 15 12  6  5 16 21]
Partial sum:  [43 85 50 75]
[253  85  50  75  18   1  20  18   1   8  17  21  10   0   2  10  19   9
  15  12   6   5  16  21]
Sum:  253


### C++ code examples
- https://sodocumentation.net/cuda/topic/6566/parallel-reduction--e-g--how-to-sum-an-array-

# Vectorized

### Jit 
- https://numba.pydata.org/numba-doc/latest/cuda/ufunc.html

In [46]:
import math
from numba import vectorize, cuda
import numpy as np

@vectorize(['float32(float32)',
           'float32(float32)'],
           target='cuda')
def sigmoid(a):
    return 1/(1 + np.e ** a)
#  math.sqrt(b ** 2 - 4 * a * c)

N = 10000
dtype = np.float32

# prepare the input
A = np.array(np.random.rand(N), dtype=dtype)

sigma = sigmoid(A)

print(sigma)  # print result

[0.47867677 0.38808894 0.4059148  ... 0.2861206  0.34186673 0.30056685]
